In [6]:
import pandas as pd
from datetime import datetime
from bson.timestamp import Timestamp

In [7]:
from pymongo import MongoClient
#conn = MongoClient("mongodb://192.168.0.9",port=27017)
conn = MongoClient("mongodb://localhost",port=27017)
checklogdb = conn.dsi.logs

La problematique concerne uniquement l'entrée principale des agents.

Inclure le code de transformation des données brutes en checklog.csv

In [8]:
presences = pd.read_csv("data/checklog.csv")

def create_timestamp(line, col_date, col_time):
    datetime_str = f"{line[col_date]} {line[col_time]}"
    try:
        datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        datetime_obj = datetime.strptime(datetime_str, "%m/%d/%Y %H:%M:%S")

    return Timestamp(int(datetime_obj.timestamp()), 1).as_datetime()

def create_datetime(row):
    try:
        return datetime.strptime(row, "%Y-%m-%d")
    except ValueError:
        return datetime.strptime(row, "%m/%d/%Y")

presences["log_checkin"] = presences.apply(create_timestamp, args=("log_date","log_checkin",), axis=1)
presences["log_checkout"] = presences.apply(create_timestamp, args=("log_date","log_checkout",), axis=1)
presences["log_date"] = presences.log_date.map(lambda x: create_datetime(x))

presences.to_csv("data/out.csv")

In [11]:
#checklogdb.insert_many(presences.to_dict(orient="records"))

In [10]:
presences.head()

,log_date,log_checkin,log_checkout,log_member_id,log_member_name,log_count
0,2023-03-09,2023-03-09 00:00:00+00:00,2023-03-09 18:43:00+00:00,1210221K,KOUASSI Adrien,1
1,2023-03-09,2023-03-09 11:45:00+00:00,2023-03-09 17:02:00+00:00,1180820K,KABLAN Jean Claude,7
2,2023-03-09,2023-03-09 11:02:00+00:00,2023-03-09 11:01:00+00:00,1230521B,BEN Mourad,2
3,2023-03-09,2023-03-09 00:00:00+00:00,2023-03-09 17:20:00+00:00,2140820M,MANOUAN Nina Epse KONE,1
4,2023-03-09,2023-03-09 10:26:00+00:00,2023-03-09 12:33:00+00:00,1110720K,KOSSIKE Kango Séraphin,6
